In [1]:
import pandas as pd

# Data Inflasi

In [2]:
df = pd.read_excel('data/Data Inflasi (2000-2025).xlsx')

In [3]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,Data Inflasi,Data Inflasi,Data Inflasi,Data Inflasi
2,NaN,NaN,NaN,NaN
3,No,Periode,Data Inflasi,NaN
4,1,Maret 2025,1.03 %,NaN
...,...,...,...,...
267,264,April 2003,7.62 %,NaN
268,265,Maret 2003,7.17 %,NaN
269,266,Februari 2003,7.6 %,NaN
270,267,Januari 2003,8.68 %,NaN


In [4]:
df = df.iloc[4:].reset_index(drop=True).drop(columns=['Unnamed: 0', 'Unnamed: 3'])

In [5]:
df = df.rename(columns={'Unnamed: 1': 'Tahun', 'Unnamed: 2': 'Inflation Percentage'})

In [6]:
df

,Tahun,Inflation Percentage
0,Maret 2025,1.03 %
1,Februari 2025,-0.09 %
2,Januari 2025,0.76 %
3,Desember 2024,1.57 %
4,November 2024,1.55 %
...,...,...
263,April 2003,7.62 %
264,Maret 2003,7.17 %
265,Februari 2003,7.6 %
266,Januari 2003,8.68 %


In [7]:
# Split the Tahun column into Bulan and Tahun
df[['Bulan', 'Tahun']] = df['Tahun'].str.split(' ', expand=True)

In [8]:
df

,Tahun,Inflation Percentage,Bulan
0,2025,1.03 %,Maret
1,2025,-0.09 %,Februari
2,2025,0.76 %,Januari
3,2024,1.57 %,Desember
4,2024,1.55 %,November
...,...,...,...
263,2003,7.62 %,April
264,2003,7.17 %,Maret
265,2003,7.6 %,Februari
266,2003,8.68 %,Januari


In [9]:
df.to_csv('data-cleaned/inflation_data_cleaned.csv')

# Data Clean UMP

In [81]:
import pandas as pd
import numpy as np

def load_ump_data(start_year: int, end_year: int, folder: str = "data") -> pd.DataFrame:
    """
    Loads and cleans ump data from a CSV file with the given year range.

    Parameters:
        start_year (int): The first year in the range (inclusive).
        end_year (int): The last year in the range (inclusive).
        folder (str): Path to the folder where the CSV file is stored.

    Returns:
        pd.DataFrame: Cleaned DataFrame with numeric UMR values.
    """
    # Generate list of years
    year = list(range(start_year, end_year + 1))
    
    # Build the filename
    filename = f"{folder}/Upah Minimum Regional_Propinsi, {year[0]}-{year[-1]}.csv"

    # Read and clean the data
    df = pd.read_csv(filename)
    df = df.iloc[2:].reset_index(drop=True)

    # Rename columns
    column_renames = {'38 Provinsi': 'Provinsi'}
    for i, y in enumerate(year):
        column_renames[f'Unnamed: {i + 1}'] = y
    df.rename(columns=column_renames, inplace=True)

    # Replace "-" with NaN and convert to numeric
    df.replace("-", np.nan, inplace=True)
    for y in year:
        df[y] = pd.to_numeric(df[y], errors='coerce')

    # Fill missing values with the global mean
    global_mean = df[year].stack().mean()
    df[year] = df[year].fillna(global_mean)

    return df


In [98]:
import os
import pandas as pd
import numpy as np
import re

# Folder paths
raw_folder = "data"
clean_folder = "data-cleaned"

# Make sure the output folder exists
os.makedirs(clean_folder, exist_ok=True)

# Function to load and clean a single UMP file
def clean_umr_file(filepath, years):
    df = pd.read_csv(filepath)
    df = df.iloc[2:].reset_index(drop=True)

    # Rename columns
    column_renames = {'38 Provinsi': 'Provinsi'}
    for i, y in enumerate(years):
        column_renames[f'Unnamed: {i + 1}'] = str(y)
    df.rename(columns=column_renames, inplace=True)

    # Replace "-" with NaN and convert to numeric
    df.replace("-", np.nan, inplace=True)
    for y in years:
        df[str(y)] = pd.to_numeric(df[str(y)], errors='coerce')

    # Fill NaNs with global mean
    global_mean = df[[str(y) for y in years]].stack().mean()
    df[[str(y) for y in years]] = df[[str(y) for y in years]].fillna(global_mean)
    return df

# Loop through all CSV files in the data folder
for filename in os.listdir(raw_folder):
    match = re.search(r'(\d{4})-(\d{4})', filename)
    if match and filename.endswith(".csv"):
        start_year = int(match.group(1))
        end_year = int(match.group(2))

        # 👉 Skip if the file ends after 2014
        if end_year > 2014:
            continue

        years = list(range(start_year, end_year + 1))
        file_path = os.path.join(raw_folder, filename)
        cleaned_df = clean_umr_file(file_path, years)

        # Save cleaned DataFrame
        output_filename = f"UMP_{start_year}-{end_year}.csv"
        output_path = os.path.join(clean_folder, output_filename)
        cleaned_df.to_csv(output_path, index=False)
        print(f"✅ Saved cleaned file to: {output_path}")


✅ Saved cleaned file to: data-cleaned\UMP_2000-2002.csv
✅ Saved cleaned file to: data-cleaned\UMP_2003-2005.csv
✅ Saved cleaned file to: data-cleaned\UMP_2006-2008.csv
✅ Saved cleaned file to: data-cleaned\UMP_2009-2011.csv
✅ Saved cleaned file to: data-cleaned\UMP_2012-2014.csv


In [105]:
filename = 'data/Upah Minimum Regional_Propinsi, 2015-2018.csv'
year = [2015,2016,2018]
df = pd.read_csv(filename)
df

,38 Provinsi,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,Upah Minimum Regional/Propinsi (Rupiah),NaN,NaN
1,NaN,2015,2016,2018
2,ACEH,1900000,2118500,2700000
3,SUMATERA UTARA,1625000,1811875,2132189
4,SUMATERA BARAT,1615000,1800725,2119067
5,RIAU,1878000,2095000,2464154
6,JAMBI,1710000,1906650,2243719
7,SUMATERA SELATAN,1974346,2206000,2595995
8,BENGKULU,1500000,1605000,1888741
9,LAMPUNG,1581000,1763000,2074673


In [106]:
df = pd.read_csv(filename)
df = df.iloc[2:].reset_index(drop=True)

    # Rename columns
column_renames = {'38 Provinsi': 'Provinsi'}
for i, y in enumerate(year):
    column_renames[f'Unnamed: {i + 1}'] = y
    df.rename(columns=column_renames, inplace=True)

    # Replace "-" with NaN and convert to numeric
df.replace("-", np.nan, inplace=True)
for y in year:
    df[y] = pd.to_numeric(df[y], errors='coerce')

    # Fill missing values with the global mean
global_mean = df[year].stack().mean()
df[year] = df[year].fillna(global_mean)

df.to_csv('data-cleaned/ump_2015-2018.csv')

In [102]:
filename = 'data/Upah Minimum Regional_Propinsi, 2019-2020.csv'
year = [2019, 2020]
df = pd.read_csv(filename)
df

,38 Provinsi,Unnamed: 1,Unnamed: 2
0,NaN,Upah Minimum Regional/Propinsi (Rupiah),NaN
1,NaN,2019,2020
2,ACEH,2916810,3165031
3,SUMATERA UTARA,2303403,2499423
4,SUMATERA BARAT,2289220,2484041
5,RIAU,2662026,2888564
6,JAMBI,2423889,2630162
7,SUMATERA SELATAN,2804453,3043111
8,BENGKULU,2040407,2213604
9,LAMPUNG,2241270,2432002


In [104]:
df = pd.read_csv(filename)
df = df.iloc[2:].reset_index(drop=True)

    # Rename columns
column_renames = {'38 Provinsi': 'Provinsi'}
for i, y in enumerate(year):
    column_renames[f'Unnamed: {i + 1}'] = y
    df.rename(columns=column_renames, inplace=True)

    # Replace "-" with NaN and convert to numeric
df.replace("-", np.nan, inplace=True)
for y in year:
    df[y] = pd.to_numeric(df[y], errors='coerce')

    # Fill missing values with the global mean
global_mean = df[year].stack().mean()
df[year] = df[year].fillna(global_mean)

df.to_csv('data-cleaned/ump_2019-2020.csv')

In [109]:
import os
import pandas as pd

# Folder where the cleaned UMP files are stored
folder = "data-cleaned"

# List to store individual DataFrames
df_list = []

# Loop through files in folder
for file in os.listdir(folder):
    if file.lower().startswith("ump_") and file.endswith(".csv"):
        filepath = os.path.join(folder, file)
        df = pd.read_csv(filepath)
        df_list.append(df)

# Merge all DataFrames by columns
df_merged = pd.concat(df_list, axis=1)

# Remove duplicate 'Provinsi' columns caused by concat
# Keep the first and drop others
df_merged = df_merged.loc[:,~df_merged.columns.duplicated()]
df_merged.drop(columns=['Unnamed: 0'], inplace=True)
# Save to a single CSV
df_merged.to_csv("data-cleaned/UMP_MERGED.csv", index=False)
print("✅ Merged file saved as: data-cleaned/UMP_MERGED.csv")


✅ Merged file saved as: data-cleaned/UMP_MERGED.csv


In [110]:
df_merged

,Provinsi,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2018,2019,2020
0,ACEH,265000.000000,300000.000000,330000.000000,425000.000000,550000.000000,620000.000000,820000.000000,850000.000000,1.000000e+06,...,1.300000e+06,1.350000e+06,1.400000e+06,1.550000e+06,1.750000e+06,1.900000e+06,2.118500e+06,2.700000e+06,2.916810e+06,3.165031e+06
1,SUMATERA UTARA,254000.000000,340500.000000,464000.000000,505000.000000,537000.000000,600000.000000,737794.000000,761000.000000,8.222050e+05,...,9.650000e+05,1.035500e+06,1.200000e+06,1.375000e+06,1.505850e+06,1.625000e+06,1.811875e+06,2.132189e+06,2.303403e+06,2.499423e+06
2,SUMATERA BARAT,200000.000000,250000.000000,385000.000000,435000.000000,480000.000000,540000.000000,650000.000000,725000.000000,8.000000e+05,...,9.400000e+05,1.055000e+06,1.150000e+06,1.350000e+06,1.490000e+06,1.615000e+06,1.800725e+06,2.119067e+06,2.289220e+06,2.484041e+06
3,RIAU,250700.000000,329000.000000,394000.000000,437500.000000,476900.000000,551500.000000,637000.000000,710000.000000,8.000000e+05,...,1.016000e+06,1.120000e+06,1.238000e+06,1.400000e+06,1.700000e+06,1.878000e+06,2.095000e+06,2.464154e+06,2.662026e+06,2.888564e+06
4,JAMBI,173000.000000,245000.000000,304000.000000,390000.000000,425000.000000,485000.000000,563000.000000,658000.000000,7.240000e+05,...,9.000000e+05,1.028000e+06,1.142500e+06,1.300000e+06,1.502300e+06,1.710000e+06,1.906650e+06,2.243719e+06,2.423889e+06,2.630162e+06
5,SUMATERA SELATAN,190000.000000,255000.000000,331500.000000,403500.000000,460000.000000,503700.000000,604000.000000,662000.000000,7.430000e+05,...,9.278250e+05,1.048440e+06,1.195220e+06,1.630000e+06,1.825000e+06,1.974346e+06,2.206000e+06,2.595995e+06,2.804453e+06,3.043111e+06
6,BENGKULU,173000.000000,240000.000000,295000.000000,330000.000000,363000.000000,430000.000000,516000.000000,644838.000000,6.900000e+05,...,7.800000e+05,8.150000e+05,9.300000e+05,1.200000e+06,1.350000e+06,1.500000e+06,1.605000e+06,1.888741e+06,2.040407e+06,2.213604e+06
7,LAMPUNG,192000.000000,240000.000000,310000.000000,350000.000000,377500.000000,405000.000000,505000.000000,555000.000000,6.170000e+05,...,7.675000e+05,8.550000e+05,9.750000e+05,1.150000e+06,1.399037e+06,1.581000e+06,1.763000e+06,2.074673e+06,2.241270e+06,2.432002e+06
8,KEP. BANGKA BELITUNG,190000.000000,255000.000000,345000.000000,379500.000000,447900.000000,560000.000000,640000.000000,720000.000000,8.130000e+05,...,9.100000e+05,1.024000e+06,1.110000e+06,1.265000e+06,1.640000e+06,2.100000e+06,2.341500e+06,2.755444e+06,2.976706e+06,3.230024e+06
9,KEP. RIAU,300000.000000,421500.000000,289146.315789,460809.553191,460809.553191,557000.000000,760000.000000,805000.000000,8.330000e+05,...,9.250000e+05,9.750000e+05,1.015000e+06,1.365087e+06,1.665000e+06,1.954000e+06,2.178710e+06,2.563875e+06,2.769754e+06,3.005460e+06


In [158]:
df = pd.read_excel('data/UMP 2018-2022.xlsx')

In [159]:
df_merged

,Provinsi,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2018,2019,2020
0,ACEH,265000.000000,300000.000000,330000.000000,425000.000000,550000.000000,620000.000000,820000.000000,850000.000000,1.000000e+06,...,1.300000e+06,1.350000e+06,1.400000e+06,1.550000e+06,1.750000e+06,1.900000e+06,2.118500e+06,2.700000e+06,2.916810e+06,3.165031e+06
1,SUMATERA UTARA,254000.000000,340500.000000,464000.000000,505000.000000,537000.000000,600000.000000,737794.000000,761000.000000,8.222050e+05,...,9.650000e+05,1.035500e+06,1.200000e+06,1.375000e+06,1.505850e+06,1.625000e+06,1.811875e+06,2.132189e+06,2.303403e+06,2.499423e+06
2,SUMATERA BARAT,200000.000000,250000.000000,385000.000000,435000.000000,480000.000000,540000.000000,650000.000000,725000.000000,8.000000e+05,...,9.400000e+05,1.055000e+06,1.150000e+06,1.350000e+06,1.490000e+06,1.615000e+06,1.800725e+06,2.119067e+06,2.289220e+06,2.484041e+06
3,RIAU,250700.000000,329000.000000,394000.000000,437500.000000,476900.000000,551500.000000,637000.000000,710000.000000,8.000000e+05,...,1.016000e+06,1.120000e+06,1.238000e+06,1.400000e+06,1.700000e+06,1.878000e+06,2.095000e+06,2.464154e+06,2.662026e+06,2.888564e+06
4,JAMBI,173000.000000,245000.000000,304000.000000,390000.000000,425000.000000,485000.000000,563000.000000,658000.000000,7.240000e+05,...,9.000000e+05,1.028000e+06,1.142500e+06,1.300000e+06,1.502300e+06,1.710000e+06,1.906650e+06,2.243719e+06,2.423889e+06,2.630162e+06
5,SUMATERA SELATAN,190000.000000,255000.000000,331500.000000,403500.000000,460000.000000,503700.000000,604000.000000,662000.000000,7.430000e+05,...,9.278250e+05,1.048440e+06,1.195220e+06,1.630000e+06,1.825000e+06,1.974346e+06,2.206000e+06,2.595995e+06,2.804453e+06,3.043111e+06
6,BENGKULU,173000.000000,240000.000000,295000.000000,330000.000000,363000.000000,430000.000000,516000.000000,644838.000000,6.900000e+05,...,7.800000e+05,8.150000e+05,9.300000e+05,1.200000e+06,1.350000e+06,1.500000e+06,1.605000e+06,1.888741e+06,2.040407e+06,2.213604e+06
7,LAMPUNG,192000.000000,240000.000000,310000.000000,350000.000000,377500.000000,405000.000000,505000.000000,555000.000000,6.170000e+05,...,7.675000e+05,8.550000e+05,9.750000e+05,1.150000e+06,1.399037e+06,1.581000e+06,1.763000e+06,2.074673e+06,2.241270e+06,2.432002e+06
8,KEP. BANGKA BELITUNG,190000.000000,255000.000000,345000.000000,379500.000000,447900.000000,560000.000000,640000.000000,720000.000000,8.130000e+05,...,9.100000e+05,1.024000e+06,1.110000e+06,1.265000e+06,1.640000e+06,2.100000e+06,2.341500e+06,2.755444e+06,2.976706e+06,3.230024e+06
9,KEP. RIAU,300000.000000,421500.000000,289146.315789,460809.553191,460809.553191,557000.000000,760000.000000,805000.000000,8.330000e+05,...,9.250000e+05,9.750000e+05,1.015000e+06,1.365087e+06,1.665000e+06,1.954000e+06,2.178710e+06,2.563875e+06,2.769754e+06,3.005460e+06


In [160]:
df = df.iloc[4:39].reset_index(drop=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
df.rename(columns={'Unnamed: 1': 'Provinsi', 'Unnamed: 2': '2018', 'Unnamed: 3': '2019', 'Unnamed: 4': '2020', 'Unnamed: 5': '2021', 'Unnamed: 6': '2022'}, inplace=True)
df.replace("Rata-Rata", 'Indonesia', inplace=True)
df

,Provinsi,2018,2019,2020,2021,2022
0,Aceh,2.700000e+06,2.916810e+06,3.165031e+06,3.165031e+06,3.166460e+06
1,Sumatera Utara,2.132189e+06,2.303403e+06,2.499423e+06,2.499423e+06,2.522610e+06
2,Sumatera Barat,2.119067e+06,2.289220e+06,2.484041e+06,2.484041e+06,2.512539e+06
3,Riau,2.464154e+06,2.662026e+06,2.888564e+06,2.888564e+06,2.938564e+06
4,Jambi,2.243719e+06,2.423889e+06,2.630162e+06,2.630162e+06,2.698941e+06
5,Sumatera Selatan,2.595995e+06,2.804453e+06,3.043111e+06,3.144446e+06,3.144446e+06
6,Bengkulu,1.888741e+06,2.040407e+06,2.213604e+06,2.215000e+06,2.238094e+06
7,Lampung,2.074673e+06,2.241270e+06,2.432002e+06,2.432002e+06,2.440486e+06
8,Bangka Belitung,2.755444e+06,2.976706e+06,3.230024e+06,3.230024e+06,3.264884e+06
9,Kepulauan Riau,2.563875e+06,2.769754e+06,3.005460e+06,3.005460e+06,3.050172e+06


In [161]:
# Uppercase
df["Provinsi"] = df["Provinsi"].str.upper()
df_merged["Provinsi"] = df_merged["Provinsi"].str.upper()

In [162]:
df.drop(columns=['2018', '2019', '2020'], inplace=True)


In [165]:
df_merged = df_merged.merge(df, on='Provinsi', how='left')

In [ ]:
df_merged
global_mean = df_merged.select_dtypes(include='number').stack().mean()
df_merged.fillna(global_mean, inplace=True)
df_merged

,Provinsi,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2012,2013,2014,2015,2016,2018,2019,2020,2021,2022
0,ACEH,265000.000000,300000.000000,330000.000000,425000.000000,550000.000000,620000.000000,820000.000000,850000.000000,1.000000e+06,...,1.400000e+06,1.550000e+06,1.750000e+06,1.900000e+06,2.118500e+06,2.700000e+06,2.916810e+06,3.165031e+06,3.165031e+06,3.166460e+06
1,SUMATERA UTARA,254000.000000,340500.000000,464000.000000,505000.000000,537000.000000,600000.000000,737794.000000,761000.000000,8.222050e+05,...,1.200000e+06,1.375000e+06,1.505850e+06,1.625000e+06,1.811875e+06,2.132189e+06,2.303403e+06,2.499423e+06,2.499423e+06,2.522610e+06
2,SUMATERA BARAT,200000.000000,250000.000000,385000.000000,435000.000000,480000.000000,540000.000000,650000.000000,725000.000000,8.000000e+05,...,1.150000e+06,1.350000e+06,1.490000e+06,1.615000e+06,1.800725e+06,2.119067e+06,2.289220e+06,2.484041e+06,2.484041e+06,2.512539e+06
3,RIAU,250700.000000,329000.000000,394000.000000,437500.000000,476900.000000,551500.000000,637000.000000,710000.000000,8.000000e+05,...,1.238000e+06,1.400000e+06,1.700000e+06,1.878000e+06,2.095000e+06,2.464154e+06,2.662026e+06,2.888564e+06,2.888564e+06,2.938564e+06
4,JAMBI,173000.000000,245000.000000,304000.000000,390000.000000,425000.000000,485000.000000,563000.000000,658000.000000,7.240000e+05,...,1.142500e+06,1.300000e+06,1.502300e+06,1.710000e+06,1.906650e+06,2.243719e+06,2.423889e+06,2.630162e+06,2.630162e+06,2.698941e+06
5,SUMATERA SELATAN,190000.000000,255000.000000,331500.000000,403500.000000,460000.000000,503700.000000,604000.000000,662000.000000,7.430000e+05,...,1.195220e+06,1.630000e+06,1.825000e+06,1.974346e+06,2.206000e+06,2.595995e+06,2.804453e+06,3.043111e+06,3.144446e+06,3.144446e+06
6,BENGKULU,173000.000000,240000.000000,295000.000000,330000.000000,363000.000000,430000.000000,516000.000000,644838.000000,6.900000e+05,...,9.300000e+05,1.200000e+06,1.350000e+06,1.500000e+06,1.605000e+06,1.888741e+06,2.040407e+06,2.213604e+06,2.215000e+06,2.238094e+06
7,LAMPUNG,192000.000000,240000.000000,310000.000000,350000.000000,377500.000000,405000.000000,505000.000000,555000.000000,6.170000e+05,...,9.750000e+05,1.150000e+06,1.399037e+06,1.581000e+06,1.763000e+06,2.074673e+06,2.241270e+06,2.432002e+06,2.432002e+06,2.440486e+06
8,KEP. BANGKA BELITUNG,190000.000000,255000.000000,345000.000000,379500.000000,447900.000000,560000.000000,640000.000000,720000.000000,8.130000e+05,...,1.110000e+06,1.265000e+06,1.640000e+06,2.100000e+06,2.341500e+06,2.755444e+06,2.976706e+06,3.230024e+06,1.223113e+06,1.223113e+06
9,KEP. RIAU,300000.000000,421500.000000,289146.315789,460809.553191,460809.553191,557000.000000,760000.000000,805000.000000,8.330000e+05,...,1.015000e+06,1.365087e+06,1.665000e+06,1.954000e+06,2.178710e+06,2.563875e+06,2.769754e+06,3.005460e+06,1.223113e+06,1.223113e+06


In [198]:
df = pd.read_excel('data/UMP 2023.xlsx')


In [196]:
df = df.iloc[4:39].reset_index(drop=True).drop(columns=['Unnamed: 2', 'Unnamed: 3']).rename(columns={'Upah Minimum Provinsi ': 'Provinsi', 'Unnamed: 1': '2023'}).replace("Rata-rata", 'Indonesia')

In [197]:
df

,Unnamed: 0,Upah Minimum Provinsi di Indonesia,Unnamed: 4
0,2,Sumatera Utara,NaN
1,3,Sumatera Barat,NaN
2,4,Riau,NaN
3,5,Jambi,NaN
4,6,Sumatera Selatan,NaN
5,7,Bengkulu,NaN
6,8,Lampung,NaN
7,9,Bangka Belitung,NaN
8,10,Kepulauan Riau,NaN
9,11,DKI Jakarta,NaN


In [192]:
df['Provinsi'] = df['Provinsi'].str.upper()


In [194]:
df_merged
global_mean = df_merged.select_dtypes(include='number').stack().mean()
df_merged.fillna(global_mean, inplace=True)
df_merged

,Provinsi,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2018,2019,2020,2021,2022,2023
0,ACEH,265000.000000,300000.000000,330000.000000,425000.000000,550000.000000,620000.000000,820000.000000,850000.000000,1.000000e+06,...,1.550000e+06,1.750000e+06,1.900000e+06,2.118500e+06,2.700000e+06,2.916810e+06,3.165031e+06,3.165031e+06,3.166460e+06,3.413666e+06
1,SUMATERA UTARA,254000.000000,340500.000000,464000.000000,505000.000000,537000.000000,600000.000000,737794.000000,761000.000000,8.222050e+05,...,1.375000e+06,1.505850e+06,1.625000e+06,1.811875e+06,2.132189e+06,2.303403e+06,2.499423e+06,2.499423e+06,2.522610e+06,2.710494e+06
2,SUMATERA BARAT,200000.000000,250000.000000,385000.000000,435000.000000,480000.000000,540000.000000,650000.000000,725000.000000,8.000000e+05,...,1.350000e+06,1.490000e+06,1.615000e+06,1.800725e+06,2.119067e+06,2.289220e+06,2.484041e+06,2.484041e+06,2.512539e+06,2.742476e+06
3,RIAU,250700.000000,329000.000000,394000.000000,437500.000000,476900.000000,551500.000000,637000.000000,710000.000000,8.000000e+05,...,1.400000e+06,1.700000e+06,1.878000e+06,2.095000e+06,2.464154e+06,2.662026e+06,2.888564e+06,2.888564e+06,2.938564e+06,3.191663e+06
4,JAMBI,173000.000000,245000.000000,304000.000000,390000.000000,425000.000000,485000.000000,563000.000000,658000.000000,7.240000e+05,...,1.300000e+06,1.502300e+06,1.710000e+06,1.906650e+06,2.243719e+06,2.423889e+06,2.630162e+06,2.630162e+06,2.698941e+06,2.943033e+06
5,SUMATERA SELATAN,190000.000000,255000.000000,331500.000000,403500.000000,460000.000000,503700.000000,604000.000000,662000.000000,7.430000e+05,...,1.630000e+06,1.825000e+06,1.974346e+06,2.206000e+06,2.595995e+06,2.804453e+06,3.043111e+06,3.144446e+06,3.144446e+06,3.404177e+06
6,BENGKULU,173000.000000,240000.000000,295000.000000,330000.000000,363000.000000,430000.000000,516000.000000,644838.000000,6.900000e+05,...,1.200000e+06,1.350000e+06,1.500000e+06,1.605000e+06,1.888741e+06,2.040407e+06,2.213604e+06,2.215000e+06,2.238094e+06,2.418280e+06
7,LAMPUNG,192000.000000,240000.000000,310000.000000,350000.000000,377500.000000,405000.000000,505000.000000,555000.000000,6.170000e+05,...,1.150000e+06,1.399037e+06,1.581000e+06,1.763000e+06,2.074673e+06,2.241270e+06,2.432002e+06,2.432002e+06,2.440486e+06,2.633285e+06
8,KEP. BANGKA BELITUNG,190000.000000,255000.000000,345000.000000,379500.000000,447900.000000,560000.000000,640000.000000,720000.000000,8.130000e+05,...,1.265000e+06,1.640000e+06,2.100000e+06,2.341500e+06,2.755444e+06,2.976706e+06,3.230024e+06,1.223113e+06,1.223113e+06,1.284255e+06
9,KEP. RIAU,300000.000000,421500.000000,289146.315789,460809.553191,460809.553191,557000.000000,760000.000000,805000.000000,8.330000e+05,...,1.365087e+06,1.665000e+06,1.954000e+06,2.178710e+06,2.563875e+06,2.769754e+06,3.005460e+06,1.223113e+06,1.223113e+06,1.284255e+06


In [216]:
df = pd.read_excel('data/UMP 2024.xlsx')


In [217]:
df = df.iloc[3:42].reset_index(drop=True).drop(columns=['Unnamed: 0', 'Unnamed: 3', 'Unnamed: 4']).rename(columns={'Upah Minimum Provinsi di Indonesia': 'Provinsi', 'Unnamed: 2': '2024'}).replace("Rata-rata", 'Indonesia')
df_merged = df_merged.merge(df, on='Provinsi', how='left')

In [218]:
df_merged
global_mean = df_merged.select_dtypes(include='number').stack().mean()
df_merged.fillna(global_mean, inplace=True)
df_merged

,Provinsi,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2018,2019,2020,2021,2022,2023,2024
0,ACEH,265000.000000,300000.000000,330000.000000,425000.000000,550000.000000,620000.000000,820000.000000,850000.000000,1.000000e+06,...,1.750000e+06,1.900000e+06,2.118500e+06,2.700000e+06,2.916810e+06,3.165031e+06,3.165031e+06,3.166460e+06,3.413666e+06,1.284255e+06
1,SUMATERA UTARA,254000.000000,340500.000000,464000.000000,505000.000000,537000.000000,600000.000000,737794.000000,761000.000000,8.222050e+05,...,1.505850e+06,1.625000e+06,1.811875e+06,2.132189e+06,2.303403e+06,2.499423e+06,2.499423e+06,2.522610e+06,2.710494e+06,1.284255e+06
2,SUMATERA BARAT,200000.000000,250000.000000,385000.000000,435000.000000,480000.000000,540000.000000,650000.000000,725000.000000,8.000000e+05,...,1.490000e+06,1.615000e+06,1.800725e+06,2.119067e+06,2.289220e+06,2.484041e+06,2.484041e+06,2.512539e+06,2.742476e+06,1.284255e+06
3,RIAU,250700.000000,329000.000000,394000.000000,437500.000000,476900.000000,551500.000000,637000.000000,710000.000000,8.000000e+05,...,1.700000e+06,1.878000e+06,2.095000e+06,2.464154e+06,2.662026e+06,2.888564e+06,2.888564e+06,2.938564e+06,3.191663e+06,1.284255e+06
4,JAMBI,173000.000000,245000.000000,304000.000000,390000.000000,425000.000000,485000.000000,563000.000000,658000.000000,7.240000e+05,...,1.502300e+06,1.710000e+06,1.906650e+06,2.243719e+06,2.423889e+06,2.630162e+06,2.630162e+06,2.698941e+06,2.943033e+06,1.284255e+06
5,SUMATERA SELATAN,190000.000000,255000.000000,331500.000000,403500.000000,460000.000000,503700.000000,604000.000000,662000.000000,7.430000e+05,...,1.825000e+06,1.974346e+06,2.206000e+06,2.595995e+06,2.804453e+06,3.043111e+06,3.144446e+06,3.144446e+06,3.404177e+06,1.284255e+06
6,BENGKULU,173000.000000,240000.000000,295000.000000,330000.000000,363000.000000,430000.000000,516000.000000,644838.000000,6.900000e+05,...,1.350000e+06,1.500000e+06,1.605000e+06,1.888741e+06,2.040407e+06,2.213604e+06,2.215000e+06,2.238094e+06,2.418280e+06,1.284255e+06
7,LAMPUNG,192000.000000,240000.000000,310000.000000,350000.000000,377500.000000,405000.000000,505000.000000,555000.000000,6.170000e+05,...,1.399037e+06,1.581000e+06,1.763000e+06,2.074673e+06,2.241270e+06,2.432002e+06,2.432002e+06,2.440486e+06,2.633285e+06,1.284255e+06
8,KEP. BANGKA BELITUNG,190000.000000,255000.000000,345000.000000,379500.000000,447900.000000,560000.000000,640000.000000,720000.000000,8.130000e+05,...,1.640000e+06,2.100000e+06,2.341500e+06,2.755444e+06,2.976706e+06,3.230024e+06,1.223113e+06,1.223113e+06,1.284255e+06,1.284255e+06
9,KEP. RIAU,300000.000000,421500.000000,289146.315789,460809.553191,460809.553191,557000.000000,760000.000000,805000.000000,8.330000e+05,...,1.665000e+06,1.954000e+06,2.178710e+06,2.563875e+06,2.769754e+06,3.005460e+06,1.223113e+06,1.223113e+06,1.284255e+06,1.284255e+06


In [219]:
df_merged.to_csv('data-cleaned/UMP_2000-2024.csv', index=False)
